## Implementation using LangChain

### Installing Dependencies

In [1]:
!pip install langchain
!pip install langchain-groq
!pip install openai
!pip install tiktoken

# libraries for RAG
!pip install faiss-cpu
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

### Importing Components

In [2]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import os

### Getting Groq API Key

In [ ]:
os.environ["GROQ_API_KEY"] = "<GROQ API KEY>"

In [4]:
llm = ChatGroq(model_name="gemma2-9b-it", temperature=0)

### Task 1: Prompting an LLM


In [5]:
question = """
Extract the important information from the following email.

Hi team, just got a call from Mrs. Eleanor Vance, policy #POL458892. She reported a kitchen fire that occurred on 12th of May 2025, around 3 PM. The main damage seems to be to the oven and surrounding cabinets. She mentioned smoke damage in the kitchen and dining area too. She thinks the total damage might be around $7,500. Her contact is 555-0123. No injuries reported, thankfully.
"""

prompt_template = "Perform the following task:\n{question}"

prompt = ChatPromptTemplate.from_messages(
    [
        ("human", prompt_template)
    ]
)

chain = prompt | llm
response = chain.invoke({"question": question}).content

print(response)

Here's the important information from the email:

* **Policyholder:** Mrs. Eleanor Vance
* **Policy Number:** POL458892
* **Incident:** Kitchen fire
* **Date:** 12th of May 2025
* **Time:** Around 3 PM
* **Damage:**
    * Oven
    * Surrounding cabinets
    * Smoke damage in kitchen and dining area
* **Estimated Damage Cost:** $7,500
* **Contact Number:** 555-0123
* **Injuries:** None reported 


Let me know if you need any further information extracted! 



### Task 2: Design effective prompts


#### Implicit vs Explicit prompting


#### Zero-shot prompting

In [6]:
question = """
Claim Notification Received:
"Hi team, just got a call from Mrs. Eleanor Vance, policy #POL458892. She reported a kitchen fire that occurred on 12th of May 2025, around 3 PM. The main damage seems to be to the oven and surrounding cabinets. She mentioned smoke damage in the kitchen and dining area too. She thinks the total damage might be around $7,500. Her contact is 555-0123. No injuries reported, thankfully."

Extract the following:
- Policy Number
- Claimant Name
- Date of Loss
- Time of Loss
- Type of Loss
- Brief Description of Damage
- Estimated Loss Amount
- Injuries Reported
"""

prompt_template = "Perform the following tasks:\n{question}"

prompt = ChatPromptTemplate.from_messages(
    [
        ("human", prompt_template)
    ]
)

chain = prompt | llm
response = chain.invoke({"question": question}).content

print(response)

Here's the extracted information from the claim notification:

* **Policy Number:** POL458892
* **Claimant Name:** Mrs. Eleanor Vance
* **Date of Loss:** 12th of May 2025
* **Time of Loss:** Around 3 PM
* **Type of Loss:** Kitchen fire
* **Brief Description of Damage:**  Main damage to the oven and surrounding cabinets. Smoke damage in the kitchen and dining area.
* **Estimated Loss Amount:** $7,500
* **Injuries Reported:** No injuries 


Let me know if you need any further information extracted! 



#### Few-shot prompting


In [7]:
question = """
Claim Notification Received:
"Hi team, just got a call from Mrs. Eleanor Vance, policy #POL458892. She reported a kitchen fire that occurred on 12th of May 2025, around 3 PM. The main damage seems to be to the oven and surrounding cabinets. She mentioned smoke damage in the kitchen and dining area too. She thinks the total damage might be around $7,500. Her contact is 555-0123. No injuries reported, thankfully."

Extract the following:
- Policy Number
- Claimant Name
- Date of Loss
- Time of Loss
- Type of Loss
- Brief Description of Damage
- Estimated Loss Amount
- Injuries Reported
"""

prompt_template = '''
### Example:
Claim Notification Received:
"Email from John Sterling (policy POL77521) re: water damage at his shop. Happened sometime last night, May 10th, 2025. A pipe burst in the ceiling. Stockroom is flooded, some damage to inventory. He's not sure on the cost yet, maybe $5k-$10k? No one was there, so no injuries."

Extracted Information:
Policy Number: POL77521
Claimant Name: John Sterling
Date of Loss: May 10th, 2025
Time of Loss: Night
Type of Loss: Water damage
Brief Description of Damage: Pipe burst in ceiling, stockroom flooded, some damage to inventory
Estimated Loss Amount: $5,000 - $10,000
Injuries Reported: No

---

### Now extract from this new notification:
{question}
'''

prompt = ChatPromptTemplate.from_messages(
    [
        ("human", prompt_template)
    ]
)

chain = prompt | llm
response = chain.invoke({"question": question}).content

print(response)

Here's the extracted information:

- **Policy Number:** POL458892
- **Claimant Name:** Mrs. Eleanor Vance
- **Date of Loss:** May 12th, 2025
- **Time of Loss:** 3 PM
- **Type of Loss:** Kitchen fire
- **Brief Description of Damage:**  Damage to oven and surrounding cabinets, smoke damage in kitchen and dining area.
- **Estimated Loss Amount:** $7,500
- **Injuries Reported:** No 


Let me know if you have any other notifications you'd like me to process! 



#### Chain-of-Thoughts with zero-shot and few-shot prompting


In [8]:
question = """
Claim Notification Received:
"Hi team, just got a call from Mrs. Eleanor Vance, policy #POL458892. She reported a kitchen fire that occurred on 12th of May 2025, around 3 PM. The main damage seems to be to the oven and surrounding cabinets. She mentioned smoke damage in the kitchen and dining area too. She thinks the total damage might be around $7,500. Her contact is 555-0123. No injuries reported, thankfully."

Extract the following:
- Policy Number
- Claimant Name
- Date of Loss
- Time of Loss
- Type of Loss
- Brief Description of Damage
- Estimated Loss Amount
- Injuries Reported
"""

prompt_template = '''
### Example:
Claim Notification Received:
"Email from John Sterling (policy POL77521) re: water damage at his shop. Happened sometime last night, May 10th, 2025. A pipe burst in the ceiling. Stockroom is flooded, some damage to inventory. He's not sure on the cost yet, maybe $5k-$10k? No one was there, so no injuries."

Step-by-step reasoning:
- Date given as 10/05/2025 → interpret format as DD/MM/YYYY → convert to May 10th, 2025


Extracted Information:
Policy Number: POL77521
Claimant Name: John Sterling
Date of Loss: May 10th, 2025
Time of Loss: Night
Type of Loss: Water damage
Brief Description of Damage: Pipe burst in ceiling, stockroom flooded, some damage to inventory
Estimated Loss Amount: $5,000 - $10,000
Injuries Reported: No

---

### Now extract from this new notification:
{question}
'''

prompt = ChatPromptTemplate.from_messages(
    [
        ("human", prompt_template)
    ]
)

chain = prompt | llm
response = chain.invoke({"question": question}).content

print(response)

Here's the extracted information:

- **Policy Number:** POL458892
- **Claimant Name:** Mrs. Eleanor Vance
- **Date of Loss:** May 12th, 2025
- **Time of Loss:** 3 PM
- **Type of Loss:** Kitchen fire
- **Brief Description of Damage:**  Main damage to oven and surrounding cabinets. Smoke damage in the kitchen and dining area.
- **Estimated Loss Amount:** $7,500
- **Injuries Reported:** No 


Let me know if you have any other notifications you'd like me to process! 



#### Role prompting

In [9]:
question = """
Claim Notification Received:
"Hi team, just got a call from Mrs. Eleanor Vance, policy #POL458892. She reported a kitchen fire that occurred on 12th of May 2025, around 3 PM. The main damage seems to be to the oven and surrounding cabinets. She mentioned smoke damage in the kitchen and dining area too. She thinks the total damage might be around $7,500. Her contact is 555-0123. No injuries reported, thankfully."

Extract the following:
- Policy Number
- Claimant Name
- Date of Loss
- Time of Loss
- Type of Loss
- Brief Description of Damage
- Estimated Loss Amount
- Injuries Reported
"""

prompt_template = '''You are a professional insurance claims data extractor. Your job is to read claim notification texts and accurately extract structured information.

### Example:
Claim Notification Received:
"Email from John Sterling (policy POL77521) re: water damage at his shop. Happened sometime last night, May 10th, 2025. A pipe burst in the ceiling. Stockroom is flooded, some damage to inventory. He's not sure on the cost yet, maybe $5k-$10k? No one was there, so no injuries."

Step-by-step reasoning:
- Date given as 10/05/2025 → interpret format as DD/MM/YYYY → convert to May 10th, 2025


Extracted Information:
Policy Number: POL77521
Claimant Name: John Sterling
Date of Loss: May 10th, 2025
Time of Loss: Night
Type of Loss: Water damage
Brief Description of Damage: Pipe burst in ceiling, stockroom flooded, some damage to inventory
Estimated Loss Amount: $5,000 - $10,000
Injuries Reported: No

---

### Now extract from this new notification:
{question}
'''

prompt = ChatPromptTemplate.from_messages(
    [
        ("human", prompt_template)
    ]
)

chain = prompt | llm
response = chain.invoke({"question": question}).content

print(response)

Here's the extracted information:

- **Policy Number:** POL458892
- **Claimant Name:** Mrs. Eleanor Vance
- **Date of Loss:** May 12th, 2025
- **Time of Loss:** 3 PM
- **Type of Loss:** Kitchen fire
- **Brief Description of Damage:**  Main damage to oven and surrounding cabinets. Smoke damage in kitchen and dining area.
- **Estimated Loss Amount:** $7,500
- **Injuries Reported:** No 


Let me know if you have any other claim notifications you'd like me to process! 



#### Instruction prompting

#### Experimenting with prompt configurations


In [10]:
# remove the Mr or Mrs from the Claimant Name
question = """
Claim Notification Received:
"Hi team, just got a call from Mrs. Eleanor Vance, policy #POL458892. She reported a kitchen fire that occurred on 12th of May 2025, around 3 PM. The main damage seems to be to the oven and surrounding cabinets. She mentioned smoke damage in the kitchen and dining area too. She thinks the total damage might be around $7,500. Her contact is 555-0123. No injuries reported, thankfully."

Extract the following:
- Policy Number
- Claimant Name
- Date of Loss
- Time of Loss
- Type of Loss
- Brief Description of Damage
- Estimated Loss Amount
- Injuries Reported
"""

prompt_template = '''You are a professional insurance claims data extractor. Your job is to read claim notification texts and accurately extract structured information.

### Example:
Claim Notification Received:
"Email from John Sterling (policy POL77521) re: water damage at his shop. Happened sometime last night, May 10th, 2025. A pipe burst in the ceiling. Stockroom is flooded, some damage to inventory. He's not sure on the cost yet, maybe $5k-$10k? No one was there, so no injuries."

Step-by-step reasoning:
- Claimant Name starting with Mr. or Mrs. -> remove Mr. or Mrs.
- Date given as 10/05/2025 → interpret format as DD/MM/YYYY → convert to May 10th, 2025


Extracted Information:
Policy Number: POL77521
Claimant Name: John Sterling
Date of Loss: May 10th, 2025
Time of Loss: Night
Type of Loss: Water damage
Brief Description of Damage: Pipe burst in ceiling, stockroom flooded, some damage to inventory
Estimated Loss Amount: $5,000 - $10,000
Injuries Reported: No

---

### Now extract from this new notification:
{question}
'''

prompt = ChatPromptTemplate.from_messages(
    [
        ("human", prompt_template)
    ]
)

chain = prompt | llm
response = chain.invoke({"question": question}).content

print(response)

Here's the extracted information:

- **Policy Number:** POL458892
- **Claimant Name:** Eleanor Vance
- **Date of Loss:** May 12th, 2025
- **Time of Loss:** 3 PM
- **Type of Loss:** Kitchen fire
- **Brief Description of Damage:**  Main damage to oven and surrounding cabinets. Smoke damage in kitchen and dining area.
- **Estimated Loss Amount:** $7,500
- **Injuries Reported:** No 


Let me know if you have any other claim notifications you'd like me to process! 



### Task 3: Engineer and manage prompts


#### Comparing by modifying prompt instructions


#### Comparing with a different temperature setting


#### Comparing different models and configurations


#### Prompt management


### Task 4: Use multimodal prompts


In [12]:
from groq import Groq
import os

In [14]:
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
model_name = "meta-llama/llama-4-scout-17b-16e-instruct"

In [22]:
image_url = "https://drive.google.com/file/d/1XjuaAlPqU-lKKV7dsB-A09zlN2UCxf9i/view?usp=sharing"

In [23]:
completion = client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What is in this image?"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": image_url
                    }
                }
            ]
        },
    ],
    temperature=0.0,
    max_completion_tokens=1024,
    top_p=1,
    stream=False,
    stop=None
)

BadRequestError: Error code: 400 - {'error': {'message': 'Get "https://drive.google.com/file/d/1XjuaAlPqU-lKKV7dsB-A09zlN2UCxf9i/view?usp=sharing": dial tcp: lookup drive.google.com on 10.34.32.10:53: no such host', 'type': 'invalid_request_error'}}

In [21]:
print(completion.choices[0].message.content)

The image depicts the Golden Gate Bridge, a famous suspension bridge in San Francisco, California. The bridge is orange and spans across a body of water, with a city skyline visible in the background. The city appears to be San Francisco, with many buildings and skyscrapers visible. The water below the bridge is dark blue and has several sailboats on it. In the distance, there are mountains or hills. The overall atmosphere suggests a daytime setting with clear weather.


### Task 5: Retrieval Augmented Prompting
